In [1]:
print("hello world")

hello world


In [ ]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship


Base = declarative_base()


class Client(Base):
    __tablename__ = 'clients'
    # clé primaire
    id = Column(Integer, primary_key=True)  
    nom = Column(String)
    # relation 1-N
    commandes = relationship("Commande", back_populates="client")  


class Commande(Base):
    __tablename__ = 'commandes'
    id = Column(Integer, primary_key=True)
    montant = Column(Integer)
    # clé étrangère
    client_id = Column(Integer, ForeignKey('clients.id'))  
    client = relationship("Client", back_populates="commandes")


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# 1. Connexion à une base de données SQLite :
# L'argument 'sqlite:///ma_base_clients.db' crée un fichier 'ma_base_clients.db' 
# dans le répertoire courant s'il n'existe pas.
engine = create_engine("sqlite:///ma_base_clients.db", echo=True)

# 2. Création des tables dans la base de données
# Cette ligne regarde toutes les classes qui héritent de Base (Client, Commande) 
# et crée les tables SQL correspondantes (si elles n'existent pas déjà).
Base.metadata.create_all(engine)

# 3. Configuration de la session :
# Lie la session à notre moteur (engine) pour pouvoir interagir avec la BDD.
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
nouveau_client = Client(nom="Alice")
session.add(nouveau_client)
session.commit()


commande = Commande(montant=200, client=nouveau_client)
session.add(commande)
session.commit()

In [ ]:
# Lire toutes les commandes d'un client
client = session.query(Client).filter_by(nom="Alice").first()
print("-"*30)
print("--- Résultat de la requête ---")
for cmd in client.commandes:
    print("-"*30)
    print(cmd.montant)
    print("-"*30)

In [ ]:
commande_a_modifier = session.query(Commande).filter_by(id=1).first()

# Modifier un montant
commande_a_modifier.montant = 300
session.commit()

In [ ]:
# Lire toutes les commandes d'un client
client = session.query(Client).filter_by(nom="Alice").first()
print("-"*30)
print("--- Résultat de la requête ---")
for cmd in client.commandes:
    print("-"*30)
    print(cmd.montant)
    print("-"*30)

In [ ]:
# Supprimer une commande
session.delete(commande_a_modifier)
session.commit()

In [ ]:
# Lire toutes les commandes d'un client
client = session.query(Client).filter_by(nom="Alice").first()
print("-"*30)
print("--- Résultat de la requête ---")
for cmd in client.commandes:
    print("-"*30)
    print(cmd.montant)
    print("-"*30)

In [ ]:
# On suppose que nous avons déjà un Client existant avec id=1
nouveau_client = Client(nom="Alice")
session.add(nouveau_client)
session.commit()
# Il est préférable d'insérer le client en amont si ce n'est pas déjà fait.
client_existant = session.query(Client).filter_by(id=1).first()

if client_existant:
    # Création des nouveaux objets Commande en Python
    nouvelle_commande_1 = Commande(montant=50, client=client_existant)
    nouvelle_commande_2 = Commande(montant=150, client=client_existant)
    nouvelle_commande_3 = Commande(montant=25, client_id=client_existant.id) # On peut aussi utiliser directement le client_id

    # 1. Ajout des trois objets à la session en une seule fois (add_all)
    session.add_all([
        nouvelle_commande_1, 
        nouvelle_commande_2, 
        nouvelle_commande_3
    ])

    # 2. Le commit unique exécute toutes les commandes INSERT en base de données.
    session.commit()
    print("Trois nouvelles commandes ont été insérées en masse.")
else:
    print("Le client avec l'ID 1 n'a pas été trouvé. Veuillez d'abord l'insérer.")

In [ ]:
# Récupération de tous les objets Commande de la table
toutes_les_commandes = session.query(Commande).all()

print(f"Nombre total de commandes trouvées : {len(toutes_les_commandes)}")

# Itérer sur la liste des objets récupérés
for commande in toutes_les_commandes:
    print(f"- Commande ID: {commande.id}, Montant: {commande.montant}, Client ID: {commande.client_id}, Client nom: {commande.client.nom}")

In [ ]:
engine.dispose()

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.exc import SQLAlchemyError # Pour la gestion des erreurs

# --- 1. DÉFINITION DES MODÈLES (Base MERISE/MLD) ---
Base = declarative_base()

class Client(Base):
    __tablename__ = 'clients'
    id = Column(Integer, primary_key=True)
    nom = Column(String)
    # Relation 1-N : un Client a plusieurs Commandes
    commandes = relationship("Commande", back_populates="client", cascade="all, delete-orphan") 
    # Ajout de 'cascade' pour garantir la suppression des commandes si le client est supprimé.

    def __repr__(self):
        return f"Client(id={self.id}, nom='{self.nom}')"

class Commande(Base):
    __tablename__ = 'commandes'
    id = Column(Integer, primary_key=True)
    montant = Column(Integer)
    # Clé étrangère vers la table 'clients'
    client_id = Column(Integer, ForeignKey('clients.id'))
    # Relation N-1 : une Commande appartient à un Client
    client = relationship("Client", back_populates="commandes")

    def __repr__(self):
        return f"Commande(id={self.id}, montant={self.montant}, client_id={self.client_id})"

# --- 2. INITIALISATION ET CONNEXION À LA BASE ---

# Crée le moteur (connexion SQLite dans un fichier 'ecom.db')
engine = create_engine("sqlite:///ecom.db", echo=False) # echo=True pour voir le SQL

# Crée les tables si elles n'existent pas encore
Base.metadata.create_all(engine)

# Crée la fabrique de sessions (la 'recette' pour créer des sessions)
Session = sessionmaker(bind=engine)

# --- 3. FONCTION D'EXÉCUTION ET GESTION DE LA TRANSACTION ---

def executer_transaction():
    print("--- Démarrage de la transaction ---")
    
    # Le bloc 'with' garantit la fermeture de la session, 
    # même en cas d'erreur.
    try:
        # 1. Création de la session (entre dans le bloc 'with')
        with Session() as session:
            # Création des nouveaux objets
            client_neo = Client(nom="Neo")
            client_trinity = Client(nom="Trinity")
            
            # Neo passe deux commandes
            client_neo.commandes.extend([
                Commande(montant=120),
                Commande(montant=25)
            ])
            
            # Ajout des deux clients (et de leurs commandes liées)
            session.add_all([client_neo, client_trinity])
            
            # Exécution des INSERTs SQL
            session.commit()
            print("Client et commandes insérés avec succès.")
            
            # --- Exemple de lecture après insertion ---
            
            # Lecture de toutes les commandes du client "Neo"
            commandes_neo = (
                session.query(Commande)
                .filter(Commande.client_id == client_neo.id)
                .all()
            )
            print(f"\nCommandes de {client_neo.nom} :")
            for cmd in commandes_neo:
                print(f"  - ID Commande: {cmd.id}, Montant: {cmd.montant}")

    except SQLAlchemyError as e:
        # Si une erreur se produit (par exemple, si la BDD est inaccessible), 
        # le 'rollback' est exécuté.
        session.rollback() 
        print(f"\n❌ ERREUR DE TRANSACTION : Opération annulée. Détail: {e}")
        
    finally:
        # La session est automatiquement fermée après le bloc 'with'.
        print("\n--- Fin de la transaction (Session fermée) ---")


# --- 4. EXÉCUTION ET NETTOYAGE FINAL ---
if __name__ == "__main__":
    executer_transaction()
    
    # Fermeture définitive du pool de connexions du moteur
    engine.dispose()
    print("Ressources du moteur libérées.")